In [1]:
!pip install segmentation-models-pytorch
!pip install -U git+https://github.com/albumentations-team/albumentations
!pip install --upgrade opencv-contrib-python
!wget -c https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1XOBoRGSraP50_pS1YPB8_i8Wmw_5L-NG' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XOBoRGSraP50_pS1YPB8_i8Wmw_5L-NG" -O 'tmp' && rm -rf /tmp/cookies.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.7 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=629a77ed90bb4690621db5518aa15536cfee6a65d38bcc79c7b76b26af0a4c49
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=dc47663c54b9e577f00bcf47968c26023ae378c299f9c8f6df722eda4e3cc61e
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels
  Cloning https://github.com/albumentations-team/albumenta

In [32]:
import torch
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tqdm import tqdm

import helper

from torch.utils.data import Dataset
import os
import numpy as np
from glob import glob
import tensorflow as tf
from torch.utils.data import DataLoader
from torch import nn
import segmentation_models_pytorch as smp
from torch.nn.functional import one_hot
from segmentation_models_pytorch.losses import DiceLoss

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
if not os.path.exists('image'):
    os.makedirs('image')

In [5]:
!tar -xvf  '/content/tmp' -C '/content/image'

tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


## Download file testing enhanment

In [6]:
!wget "https://drive.google.com/uc?export=download&id=1WeP0mTjUDBt2Zx4JWO0U0xf15jwpsr6V" -O "test-im"

--2024-02-22 14:45:27--  https://drive.google.com/uc?export=download&id=1WeP0mTjUDBt2Zx4JWO0U0xf15jwpsr6V
Resolving drive.google.com (drive.google.com)... 74.125.26.100, 74.125.26.138, 74.125.26.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.26.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1WeP0mTjUDBt2Zx4JWO0U0xf15jwpsr6V&export=download [following]
--2024-02-22 14:45:27--  https://drive.usercontent.google.com/download?id=1WeP0mTjUDBt2Zx4JWO0U0xf15jwpsr6V&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.213.132, 2607:f8b0:400c:c0a::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.213.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21318708 (20M) [application/octet-stream]
Saving to: ‘test-im’

test-im             100%[===================>]  20.33M  76.0MB/s    in 0.

In [7]:
!cp "gdrive/MyDrive/test_file.zip" "test-im"

In [8]:
!unzip 'test-im' -d 'archive'

Archive:  test-im
  inflating: archive/1 (1).jpg       
  inflating: archive/1 (10).jpg      
  inflating: archive/1 (100).jpg     
  inflating: archive/1 (11).jpg      
  inflating: archive/1 (12).jpg      
  inflating: archive/1 (13).jpg      
  inflating: archive/1 (14).jpg      
  inflating: archive/1 (15).jpg      
  inflating: archive/1 (16).jpg      
  inflating: archive/1 (17).jpg      
  inflating: archive/1 (18).jpg      
  inflating: archive/1 (19).jpg      
  inflating: archive/1 (2).jpg       
  inflating: archive/1 (20).jpg      
  inflating: archive/1 (21).jpg      
  inflating: archive/1 (22).jpg      
  inflating: archive/1 (23).jpg      
  inflating: archive/1 (24).jpg      
  inflating: archive/1 (25).jpg      
  inflating: archive/1 (26).jpg      
  inflating: archive/1 (27).jpg      
  inflating: archive/1 (28).jpg      
  inflating: archive/1 (29).jpg      
  inflating: archive/1 (3).jpg       
  inflating: archive/1 (30).jpg      
  inflating: archive/1 (31).jpg 

In [9]:
!wget 'https://drive.google.com/uc?export=download&id=1-sr6XByGYKRdIDuS3MjAWCBnhX1_OnGG' -O "test-mark"

--2024-02-22 14:45:31--  https://drive.google.com/uc?export=download&id=1-sr6XByGYKRdIDuS3MjAWCBnhX1_OnGG
Resolving drive.google.com (drive.google.com)... 74.125.26.100, 74.125.26.138, 74.125.26.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.26.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1-sr6XByGYKRdIDuS3MjAWCBnhX1_OnGG&export=download [following]
--2024-02-22 14:45:31--  https://drive.usercontent.google.com/download?id=1-sr6XByGYKRdIDuS3MjAWCBnhX1_OnGG&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.213.132, 2607:f8b0:400c:c0a::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.213.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 387773 (379K) [application/octet-stream]
Saving to: ‘test-mark’

test-mark           100%[===================>] 378.68K  --.-KB/s    in 0

In [10]:
!cp "gdrive/MyDrive/test-markup.zip" "test-mark"

In [11]:
!unzip "test-mark" -d "markup"

Archive:  test-mark
  inflating: markup/1 (1).jpg        
  inflating: markup/1 (10).jpg       
  inflating: markup/1 (100).jpg      
  inflating: markup/1 (11).jpg       
  inflating: markup/1 (12).jpg       
  inflating: markup/1 (13).jpg       
  inflating: markup/1 (14).jpg       
  inflating: markup/1 (15).jpg       
  inflating: markup/1 (16).jpg       
  inflating: markup/1 (17).jpg       
  inflating: markup/1 (18).jpg       
  inflating: markup/1 (19).jpg       
  inflating: markup/1 (2).jpg        
  inflating: markup/1 (20).jpg       
  inflating: markup/1 (21).jpg       
  inflating: markup/1 (22).jpg       
  inflating: markup/1 (23).jpg       
  inflating: markup/1 (24).jpg       
  inflating: markup/1 (25).jpg       
  inflating: markup/1 (26).jpg       
  inflating: markup/1 (27).jpg       
  inflating: markup/1 (28).jpg       
  inflating: markup/1 (29).jpg       
  inflating: markup/1 (3).jpg        
  inflating: markup/1 (30).jpg       
  inflating: markup/1 (31).jpg

In [12]:
!wget "https://drive.google.com/uc?export=download&id=1K0QTK_GSyai5vNwMgaO3Kh54n5w4Sjtx" -O "face-mark"

--2024-02-22 14:45:33--  https://drive.google.com/uc?export=download&id=1K0QTK_GSyai5vNwMgaO3Kh54n5w4Sjtx
Resolving drive.google.com (drive.google.com)... 74.125.26.100, 74.125.26.138, 74.125.26.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.26.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1K0QTK_GSyai5vNwMgaO3Kh54n5w4Sjtx&export=download [following]
--2024-02-22 14:45:33--  https://drive.usercontent.google.com/download?id=1K0QTK_GSyai5vNwMgaO3Kh54n5w4Sjtx&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.213.132, 2607:f8b0:400c:c0a::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.213.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1109365 (1.1M) [application/octet-stream]
Saving to: ‘face-mark’

face-mark           100%[===================>]   1.06M  --.-KB/s    in 

In [13]:
!cp "gdrive/MyDrive/face_marker.zip" "face-mark"

In [14]:
!unzip "face-mark" -d 'face-m'

Archive:  face-mark
  inflating: face-m/1 (1).jpg        
  inflating: face-m/1 (10).jpg       
  inflating: face-m/1 (100).jpg      
  inflating: face-m/1 (11).jpg       
  inflating: face-m/1 (12).jpg       
  inflating: face-m/1 (13).jpg       
  inflating: face-m/1 (14).jpg       
  inflating: face-m/1 (15).jpg       
  inflating: face-m/1 (16).jpg       
  inflating: face-m/1 (17).jpg       
  inflating: face-m/1 (18).jpg       
  inflating: face-m/1 (19).jpg       
  inflating: face-m/1 (2).jpg        
  inflating: face-m/1 (20).jpg       
  inflating: face-m/1 (21).jpg       
  inflating: face-m/1 (22).jpg       
  inflating: face-m/1 (23).jpg       
  inflating: face-m/1 (24).jpg       
  inflating: face-m/1 (25).jpg       
  inflating: face-m/1 (26).jpg       
  inflating: face-m/1 (27).jpg       
  inflating: face-m/1 (28).jpg       
  inflating: face-m/1 (29).jpg       
  inflating: face-m/1 (3).jpg        
  inflating: face-m/1 (30).jpg       
  inflating: face-m/1 (31).jpg

## Initialize Model

In [15]:
DATA_DIR = "image/LaPa"
DEVICE = 'cpu'
ENCODER = 'mobilenet_v2'
WEIGHTS = 'imagenet'
IMAGE_SIZE = 224
num_classes = 11
BATCH_SIZE = 64

# loading model from drive

In [16]:
!gdown --id "1-MzlRCABoTDNtOCyIM37A611e2Lpjh8u" -O "best_model"

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-MzlRCABoTDNtOCyIM37A611e2Lpjh8u
To: /content/best_model
100% 17.8M/17.8M [00:00<00:00, 158MB/s]


In [39]:
class SegmentationModel(nn.Module):
  def __init__(self):
    super(SegmentationModel,self).__init__()

    self.arc = smp.DeepLabV3Plus(
        encoder_name = ENCODER,
        encoder_weights = WEIGHTS,
        in_channels = 3,
        classes = num_classes,
        activation = None
    )
    for param in self.arc.encoder.parameters():
        param.requires_grad = False
  def forward(self, images, masks = None):
    logits = self.arc(images)

    if masks != None:
      loss1 = DiceLoss(mode="multiclass")(logits,masks)
      loss2 = nn.CrossEntropyLoss()(logits,masks)
      return logits, loss1 + loss2


    return logits

In [40]:
model = SegmentationModel()
model.to(DEVICE)


SegmentationModel(
  (arc): DeepLabV3Plus(
    (encoder): MobileNetV2Encoder(
      (features): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): InvertedResidual(
          (conv): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU6(inplace=True)
            )
            (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (2): InvertedResidual(
          (conv): Sequential(
      

# load directly from my drive or you can load model parameter from disk in colab if !wget work

In [41]:
model_name =  "model4"

In [42]:
model.load_state_dict(torch.load(f'gdrive/MyDrive/{model_name}.pt',map_location=torch.device(DEVICE)))

<All keys matched successfully>

In [43]:
from torchsummary import summary
summary(model, (3, IMAGE_SIZE, IMAGE_SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

## custom function

In [44]:
def show_image(image,mask,pred_image = None):

    if pred_image == None:

        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

        ax1.set_title('IMAGE')
        ax1.imshow(image.permute(1,2,0).squeeze(),cmap = 'gray')

        ax2.set_title('GROUND TRUTH')
        grayscale_images = torch.Tensor(np.argmax(mask.numpy(), axis=0) * (255 / (mask.shape[0] - 1)))
        ax2.imshow(grayscale_images.permute(0,1).squeeze(),cmap = 'gray')

    elif pred_image != None :

        f, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize=(10,5))

        ax1.set_title('IMAGE')
        ax1.imshow(image.permute(1,2,0).squeeze(),cmap = 'gray')

        ax2.set_title('GROUND TRUTH')
        grayscale_images = torch.Tensor(np.argmax(mask.numpy(), axis=0) * (255 / (mask.shape[0] - 1)))
        ax2.imshow(grayscale_images.permute(0,1).squeeze(),cmap = 'gray')

        ax3.set_title('MODEL OUTPUT')
        pred_image = torch.Tensor(np.argmax(pred_image.numpy(), axis=0) * (255 / (pred_image.shape[0] - 1)))
        ax3.imshow(pred_image.permute(0,1).squeeze(),cmap = 'gray')

## Evaluate

In [45]:
model.eval()

SegmentationModel(
  (arc): DeepLabV3Plus(
    (encoder): MobileNetV2Encoder(
      (features): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): InvertedResidual(
          (conv): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU6(inplace=True)
            )
            (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (2): InvertedResidual(
          (conv): Sequential(
      

class 0 = background

class 1 = skin

class 2 = left brown

class 3 = right brown

class 4 = left eye

class 5 = right eye

class 6 = nose

class 7 = upper lip

class 8 = mouth

class 9 = lower lip

class 10 = hair

## Download shape_predictor

In [24]:
import dlib

In [25]:
!wget https://github.com/italojs/facial-landmarks-recognition/archive/refs/heads/master.zip -O 'shape'

--2024-02-22 14:45:53--  https://github.com/italojs/facial-landmarks-recognition/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/italojs/facial-landmarks-recognition/zip/refs/heads/master [following]
--2024-02-22 14:45:53--  https://codeload.github.com/italojs/facial-landmarks-recognition/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘shape’

shape                   [   <=>              ]  68.98M   165MB/s    in 0.4s    

2024-02-22 14:46:03 (165 MB/s) - ‘shape’ saved [72327196]



In [26]:
!unzip 'shape' -d 'shape_pred'

Archive:  shape
d37b6a7426e98094e28fa99254e270a3e9b6d591
   creating: shape_pred/facial-landmarks-recognition-master/
  inflating: shape_pred/facial-landmarks-recognition-master/README.md  
  inflating: shape_pred/facial-landmarks-recognition-master/main.py  
  inflating: shape_pred/facial-landmarks-recognition-master/shape_predictor_68_face_landmarks.dat  


In [27]:
predictor_path = '/content/shape_pred/facial-landmarks-recognition-master/shape_predictor_68_face_landmarks.dat'

In [28]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

## Function

In [46]:
total = list(range(0,68))
left_eye_indices = list(range(36, 42))
right_eye_indices = list(range(42, 48))
mouth_indices = list(range(48, 61))
smile_indices = list(range(48, 68))

#PARAM
#use BGR which is blue / green / red
left_eye_color = [125,125,0]
#use BGR which is blue / green / red
right_eye_color = [0,125,125]
#use HSV which is hue value
mouth_h = 255

#change/swap part of image
def add_part_image(full_img,part,mask):
    mask_inv = cv2.bitwise_not(mask)
    background = cv2.bitwise_and(full_img,full_img,mask = mask_inv)
    result = cv2.add(background,part)
    return result

#change mouth tone color base on mask (if it black or white tone it won't change)
def change_mouth_color(bgr_img,mask,hue,logit_mouth,CURRENT_FACIAL):
    hsv_img = cv2.cvtColor(bgr_img,cv2.COLOR_BGR2HSV)
    my_mask = None
    if mask is not None:
        _,thresh = cv2.threshold(mask, 0 , 255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        if (CURRENT_FACIAL is not None):
            CURRENT_FACIAL = cv2.bitwise_or(CURRENT_FACIAL,thresh)
        my_mask = cv2.bitwise_or(logit_mouth,mask)
    else:
        my_mask = logit_mouth
    h, s, v = cv2.split(hsv_img)
    h[my_mask == 255] = hue
    hsv_img = cv2.merge([h,s,v])

    img = cv2.cvtColor(hsv_img,cv2.COLOR_HSV2BGR)
    return img , CURRENT_FACIAL

#change iris eye base on mask
def change_iris_eye(bgr_img,mask,color,CURRENT_FACIAL):
    gray = cv2.cvtColor(bgr_img,cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_and(gray,gray,mask = mask)
   # eye = cv2.bitwise_and(gray,gray,mask = mask)
    _,thresh = cv2.threshold(gray, 0 , 255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh = cv2.bitwise_and(thresh,thresh,mask = mask)

    iris = bgr_img.copy()
    iris[:,:] = color

    iris = cv2.bitwise_and(iris,iris,mask = thresh)
    if (CURRENT_FACIAL is not  None):
        CURRENT_FACIAL = cv2.add(CURRENT_FACIAL,thresh)
    #iris = cv2.bitwise_and(iris,iris,mask = mask)

    result = add_part_image(bgr_img,iris,thresh)
    return result , CURRENT_FACIAL

#get face mask
def get_face(gray,landmarks):
    mask = np.zeros_like(gray)
    points = np.array([(landmarks.part(n).x,landmarks.part(n).y) for n in total])
    hull = cv2.convexHull(points)
    cv2.fillConvexPoly(mask, hull,255)
    return mask

#enhance face with mask
def enhance_face(hsv_img,result,mask):
    face_region = cv2.bitwise_and(hsv_img,hsv_img, mask = mask)
    face_region[:,:,2] = cv2.equalizeHist(face_region[:,:,2])
    face_region = cv2.cvtColor(face_region,cv2.COLOR_HSV2BGR)
    result = add_part_image(result,face_region,mask)
    return result

#get mouth mask
def get_mouth(logit,WIDTH,HEIGHT):
    mouth0 = (logit.numpy().astype(np.uint8))[7,:,:]
    mouth1 = (logit.numpy().astype(np.uint8))[8,:,:]
    mouth2 = (logit.numpy().astype(np.uint8))[9,:,:]

    _, mouth0 =  cv2.threshold(mouth0, 0 , 255,cv2.THRESH_BINARY + cv2.THRESH_OTSU )
    _, mouth1 =  cv2.threshold(mouth1, 0 , 255,cv2.THRESH_BINARY + cv2.THRESH_OTSU )
    _, mouth2 =  cv2.threshold(mouth2, 0 , 255,cv2.THRESH_BINARY + cv2.THRESH_OTSU )

    mouth = mouth0
    mouth = cv2.bitwise_or(mouth,mouth1)
    mouth = cv2.bitwise_or(mouth,mouth2)

    kernel = np.ones((5,5),np.uint8)
    mouth = cv2.morphologyEx(mouth,cv2.MORPH_OPEN,kernel,iterations = 1)
    mouth = cv2.resize(mouth,(WIDTH,HEIGHT))

    return mouth


# combination of function to enhancement
def main(detector,predictor,frame,logit,logit_mouth,CURRENT_FACIAL,CURRENT_FACE):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    result = frame.copy()
    hsv_img = cv2.cvtColor(result,cv2.COLOR_BGR2HSV)
    B,G,R = cv2.split(result)
    laplacian = np.array([
        [0,-1,0],
        [-1,5,-1],
        [0,-1,0]
    ])
    blur_kernel = (5,5)
    B = cv2.GaussianBlur(B,blur_kernel,0)
    G = cv2.GaussianBlur(G,blur_kernel,0)
    R = cv2.GaussianBlur(R,blur_kernel,0)

    R = cv2.medianBlur(R,3)
    G = cv2.medianBlur(G,3)
    B = cv2.medianBlur(B,3)

    B = cv2.filter2D(B.astype(np.int16),-1,laplacian).clip(0,255).astype(np.uint8)
    G = cv2.filter2D(G.astype(np.int16),-1,laplacian).clip(0,255).astype(np.uint8)
    R = cv2.filter2D(R.astype(np.int16),-1,laplacian).clip(0,255).astype(np.uint8)
    result = cv2.merge([B,G,R])
    for face in faces:
        landmarks = predictor(gray, face)
        mask = get_face(gray,landmarks)
        if (CURRENT_FACE is not None):
            CURRENT_FACE = cv2.bitwise_or(CURRENT_FACE,mask)
        result = enhance_face(hsv_img,result,cv2.bitwise_or(mask,logit))
        left_eye = np.array([((landmarks.part(idx).x) , (landmarks.part(idx).y)) for idx in left_eye_indices])
        right_eye = np.array([((landmarks.part(idx).x) , (landmarks.part(idx).y)) for idx in right_eye_indices])
        mouth = np.array([((landmarks.part(idx).x) , (landmarks.part(idx).y)) for idx in mouth_indices])
        mask1 = np.zeros(frame.shape[:2],dtype=np.uint8)
        mask2 = np.zeros(frame.shape[:2],dtype=np.uint8)
        mask3 = np.zeros(frame.shape[:2],dtype=np.uint8)
        cv2.drawContours(mask1,[left_eye],-1, (255, 255, 255), -1, cv2.LINE_AA)
        cv2.drawContours(mask2,[right_eye],-1, (255, 255, 255), -1, cv2.LINE_AA)
        cv2.drawContours(mask3,[mouth],-1, (255, 255, 255), -1, cv2.LINE_AA)

        result , CURRENT_FACIAL = change_mouth_color(result,mask3,mouth_h,logit_mouth,CURRENT_FACIAL)
        result , CURRENT_FACIAL = change_iris_eye(result,mask1,left_eye_color,CURRENT_FACIAL)
        result , CURRENT_FACIAL = change_iris_eye(result,mask2,right_eye_color,CURRENT_FACIAL)
    if (len(faces) == 0):
        result = enhance_face(hsv_img,result,logit)
        result , CURRENT_FACIAL = change_mouth_color(result,None,mouth_h,logit_mouth,CURRENT_FACIAL)
    return result , CURRENT_FACIAL , CURRENT_FACE

#
def enhancer(image):
    HEIGHT,WIDTH = image.shape[:2]


    #initial combine result mask from method
    # get mask from model
    my_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    my_img = cv2.resize(my_img,(224,224))
    my_img = np.transpose(my_img, (2,0,1)).astype(np.float32)
    my_img = torch.Tensor(my_img) / 255.0
    logit = model(my_img.to(DEVICE).unsqueeze(0))
    my_logit = logit.detach().cpu().squeeze(0)
    my_logit = torch.softmax(my_logit,dim = 0)
    my_logit = torch.argmax(my_logit, dim=0)

    my_logit = one_hot(my_logit,11)
    my_logit = my_logit.permute((2,0,1)) * 255.0

    #get face
    mouth = get_mouth(my_logit,WIDTH,HEIGHT)
    nhair = (my_logit.numpy().astype(np.uint8))[10,:,:]
    face = (my_logit.numpy().astype(np.uint8))[0,:,:]
    _, nhair =  cv2.threshold(nhair, 0 , 255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU )
    _, face =  cv2.threshold(face, 0 , 255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU )
    mask_face = cv2.bitwise_and(nhair,face)
    kernel = np.ones((5,5),np.uint8)
    #remove noise
    mask_face = cv2.morphologyEx(mask_face,cv2.MORPH_OPEN,kernel,iterations = 1)
    mask_face = cv2.resize(mask_face,(WIDTH,HEIGHT))
    # result = main(detector,predictor,image)(detector,predictor,frame,logit,logit_mouth,CURRENT_FACIAL,CURRENT_FACE)
    result , CURRENT_FACIAL , CURRENT_FACE = main(detector,predictor,image,mask_face,mouth,None, None)
    result = cv2.cvtColor(result,cv2.COLOR_BGR2RGB)

    return result

## Use Case

In [47]:
def my_iou(predict,mask):
    predict = predict // 255
    mask = mask // 255
    intersection = np.sum(predict & mask)
    union = np.sum(predict | mask)
    return (intersection + 1e-10) / (union + 1e-10)

### Eval

In [48]:
path_img = '/content/archive'
path_markup = '/content/markup'
path_face_m = '/content/face-m'
score = np.zeros(100,dtype = np.float64)
fscore = np.zeros(100,dtype = np.float64)
psnr = np.zeros(100,dtype = np.float64)
c = 10
K = 0.0001
model.eval()
with torch.no_grad():
    for i in range(0,100):
        p = os.path.join(path_img,f'1 ({i + 1}).jpg')
        image = cv2.imread(p)
        HEIGHT,WIDTH = image.shape[:2]
        my_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        my_img = cv2.resize(my_img,(224,224))
        my_img = np.transpose(my_img, (2,0,1)).astype(np.float32)
        my_img = torch.Tensor(my_img) / 255.0
        logit = model(my_img.to(DEVICE).unsqueeze(0))

        CURRENT_FACIAL = np.zeros((HEIGHT,WIDTH),dtype = np.uint8)
        CURRENT_FACE = np.zeros((HEIGHT,WIDTH),dtype = np.uint8)

        my_logit = logit.detach().cpu().squeeze(0)
        my_logit = torch.softmax(my_logit, dim = 0)
        my_logit = torch.argmax(my_logit, dim=0)

        my_logit = one_hot(my_logit,11)
        my_logit = my_logit.permute((2,0,1)) * 255.0
       # my_logit = (my_logit > 0.5) * 255.0

        mouth = get_mouth(my_logit,WIDTH,HEIGHT)
        # get hair -> inv -> get all area that doesn't include hair
        nhair = (my_logit.numpy().astype(np.uint8))[10,:,:]
        # background -> inv -> get all area that not background(head of person)
        # when two mask use and bitwise will got all face that don't include hair
        face = (my_logit.numpy().astype(np.uint8))[0,:,:]
        # not hair will get
        _, nhair =  cv2.threshold(nhair, 0 , 255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU )
        # not background will get head area for all
        _, face =  cv2.threshold(face, 0 , 255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU )
        mask_face = cv2.bitwise_and(nhair,face)
        kernel = np.ones((5,5),np.uint8)
        mask_face = cv2.morphologyEx(mask_face,cv2.MORPH_OPEN,kernel,iterations = 1)
        mask_face = cv2.resize(mask_face,(WIDTH,HEIGHT))

        p_markup = os.path.join(path_markup,f'1 ({i + 1}).jpg')
        markup = cv2.imread(p_markup,0)
        _, thresh =  cv2.threshold(markup, 0 , 255,cv2.THRESH_BINARY + cv2.THRESH_OTSU )

        f_markup = os.path.join(path_face_m,f'1 ({i + 1}).jpg')
        fmarkup = cv2.imread(f_markup,0)
        _, fthresh =  cv2.threshold(fmarkup, 0 , 255,cv2.THRESH_BINARY + cv2.THRESH_OTSU )
        CURRENT_FACE = mask_face
        CURRENT_FACIAL = mouth
    # result = main(detector,predictor,image)
        result , CURRENT_FACIAL , CURRENT_FACE = main(detector,predictor,image,mask_face,mouth,CURRENT_FACIAL, CURRENT_FACE)
        result = cv2.cvtColor(result,cv2.COLOR_BGR2RGB)
        score[i] = my_iou(CURRENT_FACIAL,thresh)
        fscore[i] = my_iou(CURRENT_FACE,fthresh)
        original = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        print(f'facial enhance = {score[i]} , face enhance = { fscore[i]}')
        plt.figure(figsize=(14,6))
        plt.subplot(1,6,1)
        plt.imshow(original)
        plt.subplot(1,6,2)
        plt.imshow(result)
        plt.subplot(1,6,3)
        plt.imshow(CURRENT_FACIAL,cmap = 'gray')
        plt.subplot(1,6,4)
        plt.imshow(thresh, cmap = 'gray')
        plt.subplot(1,6,5)
        plt.imshow(CURRENT_FACE,cmap = 'gray')
        plt.subplot(1,6,6)
        plt.imshow(fthresh,cmap = 'gray')
        plt.show()
print(f'average iou = {np.mean(fscore)}')
print(f'average iou = {np.mean(score)}')

Output hidden; open in https://colab.research.google.com to view.

# Save test result

In [49]:
test = pd.DataFrame([np.mean(fscore),np.mean(score)],index = ["Face enhancement","Facial change color"],columns = ["mIoU score"])
test.to_csv(f"gdrive/MyDrive/{model_name}_result_enhancement.csv")

### Enhance random image

In [50]:
import random
path_img = '/content/archive'
path_markup = '/content/markup'
path_face_m = '/content/face-m'
c = 10
K = 0.0001
model.eval()
with torch.no_grad():
    for i in random.sample(range(0,100),5):
        p = os.path.join(path_img,f'1 ({i + 1}).jpg')
        image = cv2.imread(p)
        result = enhancer(image)

        original = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.figure(figsize=(14,6))
        plt.subplot(1,2,1)
        plt.imshow(original)
        plt.subplot(1,2,2)
        plt.imshow(result)
        plt.show()

Output hidden; open in https://colab.research.google.com to view.